In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days/project_2_news_classf_1')
print(os.getcwd())

/content
/content/drive/MyDrive/python_training/NLP100Days/project_2_news_classf_1


In [3]:
!ls

news_clustering_test.tsv   project_2_分類器_基礎篇_hw.ipynb
news_clustering_train.tsv


In [4]:
!python --version

Python 3.7.10


In [5]:
!pip install -q -U pip
!pip install -q numpy
!pip install -q pandas
!pip install -q ckiptagger
!pip install -q tqdm
!pip install -q tensorflow==1.14.0
!pip install -q ipywidgets

     |████████████████████████████████| 1.5MB 7.1MB/s 
     |████████████████████████████████| 109.3 MB 46 kB/s 
     |████████████████████████████████| 3.1 MB 47.5 MB/s 
     |████████████████████████████████| 488 kB 48.3 MB/s 
     |████████████████████████████████| 50 kB 5.3 MB/s 


In [6]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [7]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [8]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [9]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [11]:
train_titles[1]

'9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜'

In [12]:
train_classes[1]

'體育'

In [10]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞(WS) + POS

In [17]:
ws = WS('../ckip_data/')
pos = POS('../ckip_data/')

Exception ignored in: <function WS.__del__ at 0x7f1b9f62ac20>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/ckiptagger/api.py", line 65, in __del__
    self.model.sess.close()
AttributeError: 'WS' object has no attribute 'model'


In [18]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    # YOUR CODE HERE
    word_s = ws(
      title,
      sentence_segmentation = True, # To consider delimiters
      segment_delimiter_set = {",", "。", ":", "?", "!", ";"}) # This is the defualt set of delimiters
    word_p = pos(title)
    # END YOUR CODE
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [19]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    # YOUR CODE HERE
    word_s = ws(
      title,
      sentence_segmentation = True, # To consider delimiters
      segment_delimiter_set = {",", "。", ":", "?", "!", ";"}) # This is the defualt set of delimiters
    word_p = pos(title)
    # END YOUR CODE
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [20]:
train_title_cuts[120]

[('國', 'Nc')]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# Bag of Words (BOW)

In [ ]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
# YOUR CODE HERE

# END YOUR CODE

In [ ]:
word2index['溫暖']

1520

In [ ]:
index2word[1520]

'溫暖'

In [ ]:
def get_bow_vector(pairs, word2index):
    # YOUR CODE HERE

    # END YOUR CODE
    return vector

In [ ]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [ ]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['周雲傑', '鎧單', '拜仁續', '中超', '蘇群', '王楚欽', '海富通', '梅西', '王', '阿迪薩亞', '景泰石', '盧克肖', '哈希', '綠心', '馬卡報', '韋神', '·龜茲', '海爾', '楊帆', '泰森', '盼保', '陶華碧', '馬化騰', '羅永浩', '2018世界杯', '余則成', '陳柯楓', 'V10', '拳皇', '劉詩雯', '泰拳王西', '卡拉黑', 'G7', '峽谷里', '龐氏', '馬龍', '張', '群里', '劉代全', '切爾西', '惠若琪', '魅族', '奧拉朱旺', '韋德', '宇通', '維韋杜詹霍', '寶二', 'NBA', '黃紅元', '周琦', '堯治河', '吉尼斯', '纏中', '夢琪', '刷安徒恩', '萊昂納德', '清華', '旭旭', 'WWE', '宋清輝', '艷武漢', '林員', '微眾', '巴菲特', '昌圖', '老詹', '白起', '余小C', 'P60八核讓', '本澤馬', '安永', '王者榮耀', '芒格', '李謹詺', '瑞和寶', '杜蕾斯', '黃紫昌', '內馬爾', '馬夏爾', '伊亞', '綠軍', '唐培科', '高傭', '曼薩諾', '泰倫盧成', '張大仙', '力帆', '李大霄', '黃希揚', '烏迪內斯', '美宇', '霍金斯', '拉卡拉', '德羅巴', '大豐', '郭春林', '伯克希爾', '王者榮', '電一', '阿里舜宇']
VC : ['封禁', '打出', '誤判', '取代', '審查', '運算', '撒', '給出', '改', '炒', '逗', '拿', '提高到', '打爆', '浪費', '放流', '募', '構建', '種養', '去完', '買錯', '刷新', '召喚', '決殺', '撫摸', '處置', '攻略', '支持', '收穫', '集齊', '嫁', '掌握', '共進', '揭密', '進化到', '交', '大修', '購', '攔', '領導', '通', '覽', '計算', '直播', '前瞻', '對', '騎', '駕駛', '改種', '罵', '把握住', '簽', '扣', '回應', '爆

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [ ]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        # YOUR CODE HERE

        # END YOUR CODE
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [ ]:
def cosine_similarity(bow1, bow2):
    # YOUR CODE HERE

    # END YOUR CODE
    return similarity

In [ ]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [ ]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [ ]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [ ]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0.04      , 0.01333333, 0.00333333, ..., 0.        , 0.        ,
        0.        ]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333])}

# Group mean vector: 測試

In [ ]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [ ]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 73, '遊戲': 8, '財經': 6, '旅遊': 6, '農業': 6, '科技': 3})
predict 財經 :  Counter({'財經': 67, '科技': 18, '體育': 6, '旅遊': 6, '遊戲': 6, '農業': 5})
predict 科技 :  Counter({'科技': 62, '財經': 13, '體育': 10, '農業': 6, '遊戲': 6, '旅遊': 2})
predict 旅遊 :  Counter({'旅遊': 70, '農業': 11, '財經': 6, '科技': 3, '遊戲': 2})
predict 農業 :  Counter({'農業': 70, '旅遊': 9, '科技': 6, '體育': 5, '財經': 4, '遊戲': 3})
predict 遊戲 :  Counter({'遊戲': 75, '科技': 7, '旅遊': 5, '體育': 4, '財經': 4, '農業': 1})
